# NBA Player of week analysis Using Python

#### Using the NBA player dataset to capture and visualize data and insights.

Import the necessary libraries inclusing pandas, numpy and seaborn for visualization

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import datetime
from time import gmtime, strftime

%matplotlib inline

In [ ]:
nba_df = pd.read_csv("../input/NBA_player_of_the_week.csv")

Create a abbr_to_num dictionary for the month name and month value from calendar module

In [ ]:
abbr_to_num = {name: num for num, name in enumerate(calendar.month_abbr) if num}

### Segregate the team into "East" and "West" teams
Find the unique teamnames for each conference

In [ ]:
east_teams = nba_df[nba_df['Conference'] == 'East']['Team'].unique()

In [ ]:
west_teams = nba_df[nba_df['Conference'] == 'West']['Team'].unique()

## Populate Conference values

In [ ]:
def missing_conf(myteam,myConf):
    if myConf is not None :
        if ((myteam in east_teams) | (myteam == 'Washington Bullets')):
            return 'East'
        elif ((myteam in west_teams) | (myteam == 'Washington Bullets')):
            return 'West'
    else:
        return myConf

In [ ]:
nba_df['Conference_2'] = nba_df.apply(lambda row : missing_conf(row['Team'],row['Conference']), axis = 1)

In [ ]:
nba_df.head()

## Calculate BMI

In [ ]:
def makeHeightNum(x):
    if 'cm' in x:
        return int(x.replace('cm',''))
    else:
        foot = int(x.split('-')[0])
        inches = int(x.split('-')[1])
        return ((foot*12 + inches)*2.54)

In [ ]:
def makeWeightNum(x):
    if 'kg' in x:
        return int(x.replace('kg',''))
    else:
        return (int(x)*0.453592)

In [ ]:
nba_df['Height_cm'] = nba_df.apply(lambda row : makeHeightNum(row['Height']),axis = 1)

In [ ]:
nba_df['Weight_kg'] = nba_df.apply(lambda row : makeWeightNum(row['Weight']),axis = 1)

In [ ]:
nba_df.head()

In [ ]:
nba_df['BMI'] = nba_df['Weight_kg']/((nba_df['Height_cm']/100)**2)

In [ ]:
nba_df.head()

Create list of position variable, position full name and week names 

In [ ]:
pos = ["PG","SG","F","C" ,"SF","PF","G","FC","GF","F-C","G-F"]
posNames = ["point guard","shooting guard","forward","center","small forward","power forward", "guard", "forward center","guard forward", "forward center", "guard forward"]
weekday = ["Mon", "Tue", "Wed", "Thu","Fri", "Sat", "Sun"]

## Define functions for position name generation, date field manipulation and week number generation

In [ ]:
def get_position(y):
    pos_index = pos.index(y)
    return posNames[pos_index]

In [ ]:
def makeDate(x):
    current_date = x.split(',')[0]
    current_year = x.split(',')[1]
    current_day = current_date.split(' ')[1]
    current_month = abbr_to_num[current_date.split(' ')[0]]
    return (current_year.strip()+'-'+str(current_month)+'-'+current_day)

In [ ]:
def getWeek(x):
    date_ymd = makeDate(x)
    week_num = datetime.date(int(date_ymd.split('-')[0]), int(date_ymd.split('-')[1]),int(date_ymd.split('-')[2])).strftime("%V")
    return week_num

In [ ]:
def getWeekDay(x):
    date_ymd = makeDate(x)
    week_day = datetime.date(int(date_ymd.split('-')[0]), int(date_ymd.split('-')[1]),int(date_ymd.split('-')[2])).weekday()
    weekday_name = weekday[week_day]
    return weekday_name

## Apply the function on the data field

In [ ]:
nba_df['Position_name'] = nba_df.apply(lambda row : get_position(row['Position']), axis = 1)

In [ ]:
nba_df["Date_formatted"] = nba_df.apply(lambda row : makeDate(row["Date"]), axis = 1)

In [ ]:
nba_df["Week_number"] = nba_df.apply(lambda row : getWeek(row["Date"]), axis = 1)

In [ ]:
nba_df["Week_Day"] = nba_df.apply(lambda row : getWeekDay(row["Date"]), axis = 1)

In [ ]:
nba_df["Month"] = nba_df.apply(lambda row : calendar.month_name[int(row['Date_formatted'].split('-')[1])], axis = 1)

In [ ]:
nba_df.head(5)

## Visualize the parameter Height , Weight , BMI w.r.t Position

In [ ]:
plt.figure(figsize=(16,8))
plt.title('BMI v Position')
print(sns.boxplot(x='Position_name',y='BMI',data=nba_df))

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Height_cm v Position')
print(sns.boxplot(x='Position_name',y='Height_cm',data=nba_df))

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Weight_kg v Position')
print(sns.boxplot(x='Position_name',y='Weight_kg',data=nba_df))

We observe the following :
1. The Center position requires better physique in terms of Height and Weight.
2. BMI value doesn't play a major factor w.r.t position.

## Height v Weight over the count of weekly awards

In [ ]:
grouped_byPlayer = nba_df[['Real_value']].groupby(nba_df['Player']).sum()

In [ ]:
nba_df = nba_df.join(grouped_byPlayer, on='Player', how='left', lsuffix='_groupby')

In [ ]:
plt.figure(figsize = (16,8))
sns.scatterplot(x = "Height_cm", y = "Weight_kg",hue = "Position_name", size = "Real_value", data=nba_df)
sns.lineplot(x = "Height_cm", y = "Weight_kg", data = nba_df)

We observe that the center position has a very high proportion of height to weight ratio.

## Number of weekly awards w.r.t Conference

In [ ]:
grouped_by_Conference = nba_df.groupby(['Conference_2','Season short','Real_value']).sum()

In [ ]:
grouped_by_Conference.reset_index(inplace=True)

In [ ]:
plt.figure(figsize=(16,8))
sns.barplot(x='Season short',y='Real_value',hue='Conference_2',ci =None,data=grouped_by_Conference)

We see that there was a higher dominance of the West teams till the 2003 - 2004 season but later on the East teams have shown a better performance in comparison.

## Number of weekly awards w.r.t Teams

In [ ]:
grouped_by_teams = nba_df.groupby(['Team','Conference_2'])['Real_value'].sum().to_frame()

In [ ]:
grouped_by_teams.reset_index(inplace=True)

In [ ]:
plt.figure(figsize=(16,8))
g = sns.barplot(x="Team",y="Real_value",hue="Conference_2",data=grouped_by_teams)
plt.xticks(rotation=90)

Cleveland Cavaliers have the highest weekly awards count from East Conference and Los Angeles Lakers have a highest count in the West Conference.